<a href="https://colab.research.google.com/github/joaolucasbatista/Clima/blob/main/Imers%C3%A3o_Agentes_de_IA_Alura_%2B_Gemini_Ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AULA **01**

In [6]:
!pip install -q --upgrade langchain-google-genai google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.7 MB/s eta 0:00:00


Importação da API KEY

In [7]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI

Google_API_KEY = userdata.get('GEMINI_API')

Conexão com o **Gemini**

In [21]:
LLM = ChatGoogleGenerativeAI (
    model = "gemini-2.5-flash",
    temperature=1.0,
    api_key = Google_API_KEY
)

In [22]:
resp_test = LLM.invoke("Quantos dias de férias eu tenho??")
print(resp_test.content)

Como sou uma inteligência artificial, não tenho acesso aos seus dados pessoais, ao seu contrato de trabalho ou às políticas da sua empresa. Portanto, não consigo te dizer exatamente quantos dias de férias você tem.

**Para saber o número exato dos seus dias de férias, você deve verificar:**

1.  **Seu Contrato de Trabalho:** Geralmente, as condições de férias estão especificadas nele.
2.  **Departamento de Recursos Humanos (RH) ou seu Gestor:** Eles têm acesso aos seus registros e podem te informar sobre o saldo de dias de férias disponíveis.
3.  **Documentos da Empresa:** Algumas empresas têm portais online ou documentos internos onde você pode consultar essa informação.
4.  **Holerite/Recibo de Pagamento:** Às vezes, o saldo de férias é indicado no seu holerite.

**Informações gerais sobre férias no Brasil (CLT):**

*   **Regra Geral:** Após cada período de 12 meses de trabalho (período aquisitivo), o empregado tem direito a 30 dias corridos de férias.
*   **Redução dos Dias:** O núm

In [24]:
TRIAGEM_PROMPT = (
    "Você é um triador de Service Desk para políticas internas da empresa Carraro Desenvolvimento. "
    "Dada a mensagem do usuário, retorne SOMENTE um JSON com:\n"
    "{\n"
    '  "decisao": "AUTO_RESOLVER" | "PEDIR_INFO" | "ABRIR_CHAMADO",\n'
    '  "urgencia": "BAIXA" | "MEDIA" | "ALTA",\n'
    '  "campos_faltantes": ["..."]\n'
    "}\n"
    "Regras:\n"
    '- **AUTO_RESOLVER**: Perguntas claras sobre regras ou procedimentos descritos nas políticas (Ex: "Posso reembolsar a internet do meu home office?", "Como funciona a política de alimentação em viagens?").\n'
    '- **PEDIR_INFO**: Mensagens vagas ou que faltam informações para identificar o tema ou contexto (Ex: "Preciso de ajuda com uma política", "Tenho uma dúvida geral").\n'
    '- **ABRIR_CHAMADO**: Pedidos de exceção, liberação, aprovação ou acesso especial, ou quando o usuário explicitamente pede para abrir um chamado (Ex: "Quero exceção para trabalhar 5 dias remoto.", "Solicito liberação para anexos externos.", "Por favor, abra um chamado para o RH.").'
    "Analise a mensagem e decida a ação mais apropriada."
)

In [25]:
from pydantic import BaseModel, Field
from typing import Literal, List, Dict

class TriagemOut(BaseModel):
  decisao: Literal["AUTO_RESOLVER", "PEDIR_INFO", "ABRIR_CHAMADO"]
  urgencia: Literal["BAIXA", "MEDIA", "ALTA"]
  campos_faltantes: List[str] = Field(default_factory=list)

In [26]:
LLM_Triagem = ChatGoogleGenerativeAI (
    model = "gemini-2.5-flash",
    temperature=1.0,
    api_key = Google_API_KEY
)

In [27]:
from langchain_core.messages import SystemMessage, HumanMessage

Triagem_chain = LLM_Triagem.with_structured_output(TriagemOut)

def triagem(mensagem: str) -> Dict:
  saida: TriagemOut = Triagem_chain.invoke([
      SystemMessage(content=TRIAGEM_PROMPT),
      HumanMessage(content = mensagem)
      ])

  return saida.model_dump()

In [28]:
testes = ["Posso reembolsar a internet?",
          "Quero mais 5 dias de trabalho remoto, como faço?",
          "Será que posso reembolsar recursos ou treinamentos da Alura?",
          "Quantas capivaras tem no Rio Pinheiro?"]

In [29]:
for msg_teste in testes:
  print(f"Pergunta: {msg_teste}\n Resposta {triagem(msg_teste)}\n")

Pergunta: Posso reembolsar a internet?
 Resposta {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: Quero mais 5 dias de trabalho remoto, como faço?
 Resposta {'decisao': 'ABRIR_CHAMADO', 'urgencia': 'MEDIA', 'campos_faltantes': []}

Pergunta: Será que posso reembolsar recursos ou treinamentos da Alura?
 Resposta {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: Quantas capivaras tem no Rio Pinheiro?
 Resposta {'decisao': 'PEDIR_INFO', 'urgencia': 'BAIXA', 'campos_faltantes': ['Como esta pergunta se relaciona com as políticas internas da Carraro Desenvolvimento?']}



Aula **02**

In [33]:
!pip install -q --upgrade langchain_community faiss-cpu langchain_text-splitters pymupdf

In [34]:
from pathlib import Path
from langchain_community.document_loaders import PyMuPDFLoader

docs = []

for n in Path("/content/").glob("*pdf"):
  try:
    loader = PyMuPDFLoader(str(n))

    docs.extend(loader.load())
    print(f"Carregado com Sucesso {n.name}")
  except Exception as e:
    print(f"Erro ao carregar arquivo {n.name}: {e}")

print(f"Total de documentos carregados: {len(docs)}")

Carregado com Sucesso Política de Reembolsos (Viagens e Despesas).pdf
Carregado com Sucesso Política de Uso de E-mail e Segurança da Informação.pdf
Carregado com Sucesso Políticas de Home Office.pdf
Total de documentos carregados: 3


In [35]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splliter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)

chunks = splliter.split_documents(docs)

In [36]:
chunks

[Document(metadata={'producer': 'Skia/PDF m140 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': '/content/Política de Reembolsos (Viagens e Despesas).pdf', 'file_path': '/content/Política de Reembolsos (Viagens e Despesas).pdf', 'total_pages': 1, 'format': 'PDF 1.4', 'title': 'Imersão: Política de Reembolsos (Viagens e Despesas)', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 0}, page_content='Política de Reembolsos (Viagens e \nDespesas) \n \n1.\u200b Reembolso: requer nota fiscal e deve ser submetido em até 10 dias corridos após a \ndespesa.\u200b\n \n2.\u200b Alimentação em viagem: limite de R$ 70/dia por pessoa. Bebidas alcoólicas não \nsão reembolsáveis.\u200b'),
 Document(metadata={'producer': 'Skia/PDF m140 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': '/content/Política de Reembolsos (Viagens e Despesas).pdf', 'file_path': '/content/Política de Reembolsos (Viagens e 